# Concatencación y análisis de señales sintéticas

In [1]:
#Carga de paquetes 
#Core
import numpy as np
import pandas as pd
import os
from datetime import datetime
import scipy.io
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.fft import fft, fftfreq
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import Axes3D
import pylab as pl
import seaborn as sns
sns.set()
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
from numpy import mean
import statistics
from IPython.display import HTML, display_html, display
import random
import timeit

In [2]:
#Esto permite generar gráficas interactivas
%matplotlib qt

In [3]:
#Ajuste del directorio - (Al trabajar con un disco externo se requiere ajustar el directorio dependiendo del ordenador en el que se trabaje)
#Si se trabaja desde el odenador propio correr esta celda
path = 'C:\\Users\\Vaneza'
os.chdir(path)
os.getcwd()
disco = 'E'

In [ ]:
#Si se trabaja desde el ordenador del trabajo correr esta celda
path = 'C:\\Users\\Y8764982H\\OneDrive - Generalitat de Catalunya/TFG/'
os.chdir(path)
os.getcwd()
disco = 'D'

## Carga de archivos

In [4]:
#Señales reales
all_CT_PD_30s= pd.read_csv('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\segmented_tables\\all_CT_PD_30s_39ch.csv'%(disco))

In [5]:
#Carga de todas las señales generadas por SDV
files_O = []
for dirname, _, filenames in os.walk('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\\Tablas_Sinteticas_SDV\\'%(disco)):
    for filename in filenames:
        if filename.find(".csv") != -1 and filename.startswith('all_CT_PD_30s_39') and not "-checkpoint.csv" in filename:
            files_O.append(filename)
tables_O = {p: pd.read_csv('%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\\Tablas_Sinteticas_SDV\\'%(disco) + p) for p in files_O}

print(tables_O.keys())

dict_keys(['all_CT_PD_30s_39ch_10_Syn_0.csv', 'all_CT_PD_30s_39ch_10_Syn_1.csv', 'all_CT_PD_30s_39ch_10_Syn_10.csv', 'all_CT_PD_30s_39ch_10_Syn_2.csv', 'all_CT_PD_30s_39ch_10_Syn_3.csv', 'all_CT_PD_30s_39ch_10_Syn_4.csv', 'all_CT_PD_30s_39ch_10_Syn_6.csv', 'all_CT_PD_30s_39ch_10_Syn_5.csv', 'all_CT_PD_30s_39ch_10_Syn_7.csv', 'all_CT_PD_30s_39ch_10_Syn_8.csv', 'all_CT_PD_30s_39ch_10_Syn_9.csv', 'all_CT_PD_30s_39ch_10_Syn_17.csv', 'all_CT_PD_30s_39ch_10_Syn_11.csv', 'all_CT_PD_30s_39ch_10_Syn_12.csv', 'all_CT_PD_30s_39ch_10_Syn_13.csv', 'all_CT_PD_30s_39ch_10_Syn_14.csv', 'all_CT_PD_30s_39ch_10_Syn_15.csv', 'all_CT_PD_30s_39ch_10_Syn_16.csv', 'all_CT_PD_30s_39ch_10_Syn_18.csv', 'all_CT_PD_30s_39ch_10_Syn_19.csv', 'all_CT_PD_30s_39ch_10_Syn_20.csv', 'all_CT_PD_30s_39ch_10_Syn_22.csv', 'all_CT_PD_30s_39ch_10_Syn_21.csv', 'all_CT_PD_30s_39ch_10_Syn_23.csv', 'all_CT_PD_30s_39ch_10_Syn_24.csv', 'all_CT_PD_30s_39ch_10_Syn_25.csv', 'all_CT_PD_30s_39ch_10_Syn_26.csv'])


Las siguientes 3 celdas se corren después de haber genrado los archivos

In [23]:
#Señales sintéticas no filtradas
syn_NF = pd.read_csv('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_NF.csv'%(disco))

In [24]:
#Señales sintéticas filtradas
syn_F = pd.read_csv('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_NA.csv'%(disco))

In [25]:
#Señales sintéticas filtradas y aumentadas
syn_A = pd.read_csv('%s:\\TFG\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_A.csv'%(disco))

## Concatenación de las señales

In [33]:
tables_O['all_CT_PD_30s_39ch_10_Syn_0.csv']['Patient'].nunique() #Indica cuántos pacientes contiene esta tabla

10

In [ ]:
#Esta función concatena todas las señales generadas
Syn_new = tables_O['all_CT_PD_30s_39ch_10_Syn_0.csv']
for i in range(1,27):
    Syn_new = pd.concat([Syn_new,tables_O['all_CT_PD_30s_39ch_10_Syn_%i.csv'%(i)]],axis=0).reset_index(drop=True)
Syn_new

In [ ]:
#Creación de nombres de pacientes para crear ventanas de 30 segundos
for i in range(0,270):
    patient = ['Patient_%s'%(i)] * 15000
    if (i == 0):
        patient1 = pd.DataFrame(patient,columns=['Patient'])
    else:
        patient1_t = pd.DataFrame(patient,columns=['Patient'])
        patient1 = pd.concat([patient1,patient1_t],axis=0).reset_index(drop=True)
        
patient1

In [ ]:
#Cambiar la columna de tiempo de Datetime a float (volver al vector de 0 a 30s)
rng = np.arange(0.000, 30, 0.002)
rng = pd.DataFrame(rng,columns=['Time'])
#print(rng)
#print(len(rng))
Time_df = rng
for i in range(0,269):
    Time_df = pd.concat([Time_df,rng],axis=0).reset_index(drop=True)
Time_df

In [11]:
#Reemplazar las columnas de tiempo y pacientes por el nuevo vector
New_syn_table = Syn_new.copy(deep=True)
New_syn_table['Patient'] = patient1
New_syn_table['Time'] = Time_df
New_syn_table

,Patient,Diagnosis,Time,Fz,FC1,C3,T7,CP5,CP1,P3,...,POz,PO4,PO8,P6,P2,CPz,CP4,TP8,C6,C2
0,Patient_0,0,0.000,-0.003063,0.532317,0.001482,-3.656705,0.003446,1.835564,-1.143459,...,4.138678,-6.841405,0.004125,2.559477,-0.520981,-3.778777,-4.604167,0.002650,-11.473443,5.263293
1,Patient_0,0,0.002,0.384354,-9.423356,-2.123312,3.451568,0.037242,-0.264607,3.733971,...,1.034917,1.692262,-2.393040,5.534984,0.422182,-2.865445,-3.591982,0.002650,3.438419,-7.450541
2,Patient_0,0,0.004,-10.426778,-0.000855,-4.058763,-1.748821,-1.910042,1.578725,-2.354752,...,-0.694865,6.356311,-2.532005,1.142355,0.443890,-5.152515,4.297418,-2.462974,-9.122838,-7.383072
3,Patient_0,0,0.006,-3.071366,-5.627454,-2.771687,0.005552,0.137462,-2.914209,2.338753,...,2.106869,4.835685,-1.696615,0.545445,0.835161,-4.528877,-6.043903,0.002650,-8.101640,-0.004458
4,Patient_0,0,0.008,-7.063727,-6.534853,-5.473887,1.252795,0.461665,-2.900879,5.545088,...,2.345254,2.294672,-1.734597,1.076149,-1.882763,-2.659847,-2.922985,-8.698761,-9.101163,-3.242604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049995,Patient_269,0,29.990,-6.456685,0.376467,-6.761913,3.266033,2.686669,-0.851482,1.591529,...,1.836043,-7.254310,0.183805,-1.654815,-1.223988,-2.026892,-2.636632,0.002650,3.049614,-6.960283
4049996,Patient_269,0,29.992,-0.931735,-1.220577,-3.802091,2.555132,-1.004465,-3.638169,-0.663728,...,-1.472068,-3.454092,-0.188233,-4.008728,0.565905,-1.954039,5.044697,-1.525932,0.006506,-2.620820
4049997,Patient_269,0,29.994,-1.048792,-1.782919,2.049632,1.945938,-0.263482,-2.100360,2.032842,...,1.692149,-3.302905,-7.077047,4.051019,-0.316741,-7.125548,-0.012108,-3.688537,-5.642081,-5.176935
4049998,Patient_269,0,29.996,-0.144217,-0.917059,-2.721668,0.350005,5.262793,0.075380,-1.754320,...,1.477638,-2.990365,-4.924588,-4.172351,-1.260971,-1.715001,-3.955666,4.047595,-7.346419,-4.659592


In [ ]:
#Con este código se cuenta la cantidad total de pacientes de control y de párkinson
test = New_syn_table.groupby(['Diagnosis','Patient']).mean()
diag_pat = pd.DataFrame(list(test.index),columns=['Diagnosis','Patient'])
#print(diag_pat)
cont_CT = 0
for i in range(len(diag_pat)):
    if diag_pat['Diagnosis'][i] == 0:
        cont_CT = cont_CT+1

cont_PD = diag_pat['Patient'].nunique() - cont_CT
print('Pacientes de control:' ,cont_CT)     
print('Pacientes de párkinson:' ,cont_PD)

## Comprobación gráfica

In [27]:
Syn_table = syn_NF.drop(columns='Unnamed: 0')
Real_table = all_CT_PD_30s

In [ ]:
#Arreglamos tiempo en tabla real
rng = np.arange(0.000, 30, 0.002)
rng = pd.DataFrame(rng,columns=['Time'])
#print(rng)
#print(len(rng))
Time_df = rng
for i in range(0,111):
    Time_df = pd.concat([Time_df,rng],axis=0).reset_index(drop=True)
Time_df

In [ ]:
Real_table['Time'] = Time_df
Real_table

In [ ]:
# Para guardar las señales sintéticas concatenadas antes de filtrar
Syn_table.to_csv("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_NF.csv"%(disco))

In [ ]:
#Extrae la lista de nombres de los pacientes reales
Real_pat = Real_table['Patient'].drop_duplicates().reset_index(drop=True)
Real_pat = list(Real_pat)
Real_pat

In [ ]:
#Extrae la lista de nombres de los pacientes sintéticos
Syn_pat = Syn_table['Patient'].drop_duplicates().reset_index(drop=True)
Syn_pat = list(Syn_pat)
Syn_pat

### Gráficas

In [33]:
#Se escogen los pacientes aleatoreamente
CT_Pat_Syn = Syn_table[(Syn_table["Patient"] == "Patient_0")].reset_index(drop=True)
CT_Pat_Real = Real_table[(Real_table["Patient"] == "CT3_1")].reset_index(drop=True)
PD_Pat_Syn = Syn_table[(Syn_table["Patient"] == "Patient_100")].reset_index(drop=True)
PD_Pat_Real = Real_table[(Real_table["Patient"] == "PD3_1")].reset_index(drop=True)

In [34]:
#Función que organiza los datos tubulares para que funcionen adecuadamente en la el análisis
def get_record_df(df): 
    df_record = df
    df_record = df_record.drop(columns = ['Patient','Diagnosis','Time'])
    df_record = df_record.T
    return df_record.to_numpy()

In [35]:
Channels = ['Fz', 'FC1', 'C3', 'T7','CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10',
               'CP6', 'CP2', 'Cz', 'C4', 'FC2', 'F1', 'FC3', 'C1', 'C5','TP7', 'CP3', 'P1', 'P5', 'PO7',
               'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2']

In [66]:
#Gráficos reales vs. sintéticos
for i in range(0,39):
    ## Plot of CT patients 
    fig, [ax1,ax2] = plt.subplots(2,1,figsize=(20,10))
    ax1.plot(CT_Pat_Real["Time"], CT_Pat_Real[Channels[i]], label='Control %s Real'%(Channels[i]))
    ax1.plot(CT_Pat_Syn["Time"], CT_Pat_Syn[Channels[i]], label='Control %s Sintética'%(Channels[i]))
    ax1.legend(loc='upper right')
    ax1.set_ylabel("Amplitud")
    ax1.set_xlabel("Tiempo(s)")
    ax1.set_xlim(0,30)

    ## Plot of PD patients 
    ax2.plot(PD_Pat_Real["Time"], PD_Pat_Real[Channels[i]], label='Párkinson %s Real'%(Channels[i]))
    ax2.plot(PD_Pat_Syn["Time"], PD_Pat_Syn[Channels[i]], label='Párkinson %s Sintética'%(Channels[i]))
    ax2.legend(loc='upper right')
    ax2.set_ylabel("Amplitud")
    ax2.set_xlabel("Tiempo(s)")
    ax2.set_xlim(0,30)

    plt.savefig("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\Plots_real_vs_syn_NF\\CH_%s_real_vs_syn.jpg"%(disco,Channels[i]))

In [41]:
CT_signal_array_Real = get_record_df(CT_Pat_Real)
CT_signal_array_Syn = get_record_df(CT_Pat_Syn)
PD_signal_array_Real = get_record_df(PD_Pat_Real)
PD_signal_array_Syn = get_record_df(PD_Pat_Syn)

In [67]:
#Graficar la transformada de fourier de las señales reales y sintéticas
for i in range(0,39):
    # FFT _Real vs FFT _Syn
    fig, [[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,10))
    SAMPLE_RATE = 500 #Hz
    DURATION = len(CT_Pat_Real)
    N = DURATION
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    yf_CT_real = fft(CT_signal_array_Real[i])
    yf_CT_syn = fft(CT_signal_array_Syn[i])
    yf_PD_real = fft(PD_signal_array_Real[i])
    yf_PD_syn = fft(PD_signal_array_Syn[i])

    ax1.plot(xf, np.abs(yf_CT_real),label='Control %s Real'%(Channels[i]))
    ax1.set_ylim(0,15000)
    ax2.plot(xf, np.abs(yf_CT_syn),label='Control %s Sintética'%(Channels[i]))
    ax2.set_ylim(0,15000)
    ax3.plot(xf, np.abs(yf_PD_real),label='Párkinson %s Real'%(Channels[i]))
    ax3.set_ylim(0,15000)
    ax4.plot(xf, np.abs(yf_PD_syn),label='Párkinson %s Sintética'%(Channels[i]))
    ax4.set_ylim(0,15000)

    ax1.legend(loc='upper right')
    ax2.legend(loc='upper right')
    ax3.legend(loc='upper right')
    ax4.legend(loc='upper right')

    ax1.set_ylabel("Amplitud")
    ax2.set_ylabel("Amplitud")
    ax3.set_ylabel("Amplitud")
    ax4.set_ylabel("Amplitud")

    ax1.set_xlabel("Frecuencia (Hz)")
    ax2.set_xlabel("Frecuencia (Hz)")
    ax3.set_xlabel("Frecuencia (Hz)")
    ax4.set_xlabel("Frecuencia (Hz)")

    plt.savefig("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\Plots_real_vs_syn_NF\\FFT_%s_real_vs_syn.jpg"%(disco,Channels[i]))


### Filtrado


In [53]:
#Funciones de filtrado
#Cambiamos el formato de los nombres de los canales a lista
Channels = ['Fz', 'FC1', 'C3', 'T7','CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10',
               'CP6', 'CP2', 'Cz', 'C4', 'FC2', 'F1', 'FC3', 'C1', 'C5','TP7', 'CP3', 'P1', 'P5', 'PO7',
               'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2']
#Función que organiza los datos tubulares para que funcionen adecuadamente en la el análisis
def get_record_df(df): 
    df_record = df
    df_record = df_record.drop(columns = ['Patient','Diagnosis','Time'])
    df_record = df_record.T
    return df_record.to_numpy()

from scipy.signal import medfilt # import the median filter function
def median(signal,ch):# input: numpy array 1D (one column)  // ch is the number of the channel [0,63]
    array=np.array(signal)   
    #applying the median filter
    med_filtered=scipy.signal.medfilt(array[ch], kernel_size=3) # applying the median filter order3(kernel_size=3)
    return  med_filtered # return the med-filtered signal: numpy array 1D

#band pass filter between 0.5 and 40 hz
from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

#notch filter apllied at 50hz
def Implement_Notch_Filter(freq, filt_freq, Q, data):
    from scipy.signal import iirfilter
    from scipy import signal
    fs = freq # Sample frequency (Hz)
    f0 = filt_freq  # Frequency to be removed from signal (Hz)
    Q = Q  # Quality factor
    # Design notch filter
    b, a = signal.iirnotch(f0, Q, fs)
    filtered_data = lfilter(b, a, data)
    return filtered_data

In [54]:
def Signal_filtering(df,channel,frec):
    signal_array = get_record_df(df)
    #print(signal_array.shape)
    #print('signal array:', signal_array)

    med_filtered=median(signal_array,channel) # apply 3rd order median filter and store the filtred signal in med_filtred
    #print(len(med_filtered))
    #print('med_filt signal:',med_filtered)

    fs = frec
    lowcut = 0.5
    highcut = 50
    band_pass=butter_bandpass_filter(med_filtered, lowcut, highcut, fs, order=5)
    #print(len(band_pass))
    #print('band_pass signal:',band_pass)

    notch = Implement_Notch_Filter(fs, 60, 50, band_pass)
    #print(len(notch))
    #print('notch signal:',notch)
    return notch

### Graficas con filtro

In [188]:
Signal_filtering(CT_Pat_Syn,2,500)

array([ 7.89040881e-06,  6.68980734e-03,  6.54612344e-02, ...,
       -1.26481132e+00, -1.26611822e+00, -1.28257704e+00])

In [ ]:
#Señales filtradas de pacientes
for i in range(len(Channels)):
    if i == 0:
        test_signal_CT = Signal_filtering(CT_Pat_Syn,i,500)
        test_signal_PD = Signal_filtering(PD_Pat_Syn,i,500)
        test_signal_df_CT = pd.DataFrame(test_signal_CT,columns = [Channels[i]])
        test_signal_df_PD = pd.DataFrame(test_signal_PD,columns = [Channels[i]])
    else:
        test_signal1_CT = Signal_filtering(CT_Pat_Syn,i,500)
        test_signal1_PD = Signal_filtering(PD_Pat_Syn,i,500)
        test_signal1_df_CT = pd.DataFrame(test_signal1_CT,columns = [Channels[i]])
        test_signal1_df_PD = pd.DataFrame(test_signal1_PD,columns = [Channels[i]])
        test_signal_df_CT = pd.concat((test_signal_df_CT,test_signal1_df_CT), axis=1)
        test_signal_df_PD = pd.concat((test_signal_df_PD,test_signal1_df_PD), axis=1)
        
test_signal_df_PD



In [73]:
#Gráficas de señales reales vs. Sintéticas después de filtrar (Y filtrada y aumentada)
for i in range(0,39):
    ## Plot of CT patients 
    fig, [ax1,ax2] = plt.subplots(2,1,figsize=(20,10))
    ax1.plot(CT_Pat_Real["Time"], CT_Pat_Real[Channels[i]], label='Control %s Real'%(Channels[i]))
    ax1.plot(CT_Pat_Syn["Time"], test_signal_df_CT[Channels[i]]*2, label='Control %s Sintética'%(Channels[i]))  # El *2 es para duplicar la señal (Quitarlo
                                                                                                                # en caso de que se quiera solo la señal filtrada)
    ax1.legend(loc='upper right')
    ax1.set_ylabel("Amplitud")
    ax1.set_xlabel("Tiempo(s)")
    ax1.set_xlim(0,30)

    ## Plot of PD patients 
    ax2.plot(PD_Pat_Real["Time"], PD_Pat_Real[Channels[i]], label='Párkinson %s Real'%(Channels[i]))
    ax2.plot(PD_Pat_Syn["Time"], test_signal_df_PD[Channels[i]]*2, label='Párkinson %s Sintética'%(Channels[i])) # El *2 es para duplicar la señal (Quitarlo
                                                                                                                # en caso de que se quiera solo la señal filtrada)
    ax2.legend(loc='upper right')
    ax2.set_ylabel("Amplitud")
    ax2.set_xlabel("Tiempo(s)")
    ax2.set_xlim(0,30)

    plt.savefig("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\Plots_real_vs_syn_F\\CH_%s_real_vs_syn_A.jpg"%(disco,Channels[i]))


In [60]:
def get_record_df2(df): 
    df_record = df
    df_record = df_record.T
    return df_record.to_numpy()

In [61]:
CT_signal_array_Real = get_record_df(CT_Pat_Real)
CT_signal_array_Syn_F = get_record_df2(test_signal_df_CT)
PD_signal_array_Real = get_record_df(PD_Pat_Real)
PD_signal_array_Syn_F = get_record_df2(test_signal_df_PD)

In [71]:
#Graficar la transformada de fourier de las señales reales y sintéticas filtradas (Y filtradas y aumentadas)
for i in range(0,39):
    # FFT _Real vs FFT _Syn
    fig, [[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,10))
    SAMPLE_RATE = 500 #Hz
    DURATION = len(CT_Pat_Real)
    N = DURATION
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    yf_CT_real = fft(CT_signal_array_Real[i])
    yf_CT_syn = fft(CT_signal_array_Syn_F[i])*2  # El *2 es para duplicar la señal (Quitarlo en caso de que se quiera solo la señal filtrada)
    yf_PD_real = fft(PD_signal_array_Real[i])
    yf_PD_syn = fft(PD_signal_array_Syn_F[i])*2  # El *2 es para duplicar la señal (Quitarlo en caso de que se quiera solo la señal filtrada)

    ax1.plot(xf, np.abs(yf_CT_real),label='Control %s Real'%(Channels[i]))
    ax1.set_ylim(0,15000)
    ax2.plot(xf, np.abs(yf_CT_syn),label='Control %s Sintética'%(Channels[i]))
    ax2.set_ylim(0,15000)
    ax3.plot(xf, np.abs(yf_PD_real),label='Párkinson %s Real'%(Channels[i]))
    ax3.set_ylim(0,15000)
    ax4.plot(xf, np.abs(yf_PD_syn),label='Párkinson %s Sintética'%(Channels[i]))
    ax4.set_ylim(0,15000)

    ax1.legend(loc='upper right')
    ax2.legend(loc='upper right')
    ax3.legend(loc='upper right')
    ax4.legend(loc='upper right')

    ax1.set_ylabel("Amplitud")
    ax2.set_ylabel("Amplitud")
    ax3.set_ylabel("Amplitud")
    ax4.set_ylabel("Amplitud")

    ax1.set_xlabel("Frecuencia (Hz)")
    ax2.set_xlabel("Frecuencia (Hz)")
    ax3.set_xlabel("Frecuencia (Hz)")
    ax4.set_xlabel("Frecuencia (Hz)")

    plt.savefig("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\Plots_real_vs_syn_F\\FFT_%s_real_vs_syn_A.jpg"%(disco,Channels[i]))


## Código para filtrar las señales sintéticas y concatenar

In [ ]:
##Código para filtrar las señales sintéticas y concatenar

Channels = ['Fz', 'FC1', 'C3', 'T7','CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10',
            'CP6', 'CP2', 'Cz', 'C4', 'FC2', 'F1', 'FC3', 'C1', 'C5','TP7', 'CP3', 'P1', 'P5', 'PO7',
            'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2']

context = Syn_table[['Patient','Diagnosis','Time']]

for n in range(len(Syn_pat)):
    CT_or_PD_Pat_Syn = Syn_table[(Syn_table["Patient"] == Syn_pat[n])].reset_index(drop=True)
    for i in range(len(Channels)):
        if i == 0:
            test_signal = Signal_filtering(CT_or_PD_Pat_Syn,i,500)
            test_signal_df = pd.DataFrame(test_signal,columns = [Channels[i]])
        else:
            test_signal1 = Signal_filtering(CT_or_PD_Pat_Syn,i,500)
            test_signal1_df = pd.DataFrame(test_signal1,columns = [Channels[i]])
            test_signal_df = pd.concat((test_signal_df,test_signal1_df), axis=1)
    if n == 0:
        Syn_filt = test_signal_df
    else:
        Syn_filt1 = test_signal_df
        Syn_filt = pd.concat([Syn_filt,Syn_filt1],axis=0).reset_index(drop=True)                                                                          
                                                                                
                                                                                   
                                                                                   
Syn_filtered = pd.concat((context,Syn_filt),axis=1)
Syn_filtered                                                                             

In [24]:
Syn_filtered.to_csv("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_NA.csv"%(disco))

In [ ]:
##Código para filtrar las señales sintéticas, AUMENTAR y concatenar

Channels = ['Fz', 'FC1', 'C3', 'T7','CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10',
            'CP6', 'CP2', 'Cz', 'C4', 'FC2', 'F1', 'FC3', 'C1', 'C5','TP7', 'CP3', 'P1', 'P5', 'PO7',
            'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2']

context = Syn_table[['Patient','Diagnosis','Time']]

for n in range(len(Syn_pat)):
    CT_or_PD_Pat_Syn = Syn_table[(Syn_table["Patient"] == Syn_pat[n])].reset_index(drop=True)
    for i in range(len(Channels)):
        if i == 0:
            test_signal = Signal_filtering(CT_or_PD_Pat_Syn,i,500)
            test_signal_df = pd.DataFrame(test_signal*2,columns = [Channels[i]])
        else:
            test_signal1 = Signal_filtering(CT_or_PD_Pat_Syn,i,500)
            test_signal1_df = pd.DataFrame(test_signal1*2,columns = [Channels[i]])
            test_signal_df = pd.concat((test_signal_df,test_signal1_df), axis=1)
    if n == 0:
        Syn_filt = test_signal_df
    else:
        Syn_filt1 = test_signal_df
        Syn_filt = pd.concat([Syn_filt,Syn_filt1],axis=0).reset_index(drop=True)                                                                          
                                                                                
                                                                                   
                                                                                   
Syn_filtered = pd.concat((context,Syn_filt),axis=1)
Syn_filtered                                                                             

In [26]:
Syn_filtered.to_csv("%s:\\TFG\\Data and Code (1)\\Dataset\\IowaDataset\\270_CT_PD_30s_39ch_syn_A.csv"%(disco))